In [4]:
%pylab qt
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [5]:
# Loop through the ggems mhd files and save them to an array
import glob
# import os
# import numpy as np
import fastcat as fc

ggems_mhd_files = glob.glob('/home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/out/ggems_1e09_121kVp_*[!scatter].mhd')
ggems_mhd_files.sort()

ggems_projections = []

for ggems_mhd_file in ggems_mhd_files:
    ggems_projection, b, c = fc.utils.read_mhd(ggems_mhd_file)
    ggems_projections.append(ggems_projection.squeeze())

In [6]:
import mpl_interactions as pli

# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(np.array(ggems_projections), cmap='jet')

[2023-12-04 10:49:05,557] {font_manager.py:1411} DEBUG - findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
[2023-12-04 10:49:05,558] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizTwoSymReg.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
[2023-12-04 10:49:05,558] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizOneSymBol.ttf', name='STIXSizeOneSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
[2023-12-04 10:49:05,558] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts

libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [7]:
# Loop through the ggems mhd files and save them to an array
import glob
# import os
# import numpy as np
import fastcat as fc

ggems_mhd_files = glob.glob('/home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/out/ggems_1e09_121kVp_*scatter.mhd')
ggems_mhd_files.sort()

ggems_projections = []

for ggems_mhd_file in ggems_mhd_files:
    ggems_projection, b, c = fc.utils.read_mhd(ggems_mhd_file)
    ggems_projections.append(ggems_projection.squeeze())

In [8]:
import mpl_interactions as pli

# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(np.array(ggems_projections), cmap='jet',vmax=np.percentile(np.array(ggems_projections),99),vmin=np.percentile(np.array(ggems_projections),1))

In [12]:
# Use wavelet transform to find the scatter
import pywt
import numpy as np

# Define the wavelet transform
def wavelet_transform(data, wavelet='db2', level=1):
    # Create the wavelet object
    wavelet = pywt.Wavelet(wavelet)
    # Perform the wavelet transform
    coeffs = pywt.wavedec2(data, wavelet=wavelet, level=level)
    # Return the coefficients
    return coeffs

# Define the inverse wavelet transform
def inverse_wavelet_transform(coeffs, wavelet='db2'):
    # Create the wavelet object
    wavelet = pywt.Wavelet(wavelet)
    # Perform the inverse wavelet transform
    data = pywt.waverec2(coeffs, wavelet=wavelet)
    # Return the data
    return data

# Plot the wavelet transform of the ggems projections
plt.figure()
plt.imshow(np.abs(wavelet_transform(np.array(ggems_projections[0]))[0]),cmap='jet')

In [5]:
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libstdc++.so.6:${LD_PRELOAD}

In [23]:
import matplotlib.pyplot as plt

from skimage.restoration import (denoise_wavelet, estimate_sigma)
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio


original = np.mean(np.array(ggems_projections),axis=0)

sigma = 0.12
noisy = ggems_projections[0]

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 5),
                       sharex=True, sharey=True)

# tile noisy and original images with themselves 3 times over the last axis
noisy = np.concatenate([noisy[..., np.newaxis]] * 3, axis=-1).astype(np.float32)
original = np.concatenate([original[..., np.newaxis]] * 3, axis=-1).astype(np.float32)
noisy -= noisy.min()
noisy /= noisy.max()
original -= original.min()
original /= original.max()



plt.gray()

# Estimate the average noise standard deviation across color channels.
sigma_est = estimate_sigma(noisy, channel_axis=-1, average_sigmas=True) -0.07
# Due to clipping in random_noise, the estimate will be a bit smaller than the
# specified sigma.
print(f'Estimated Gaussian noise standard deviation = {sigma_est}')

im_bayes = denoise_wavelet(noisy, channel_axis=-1, convert2ycbcr=True,
                           method='BayesShrink', mode='soft',
                           rescale_sigma=True)
im_visushrink = denoise_wavelet(noisy, channel_axis=-1, convert2ycbcr=True,
                                method='VisuShrink', mode='soft',
                                sigma=sigma_est, rescale_sigma=True)

# VisuShrink is designed to eliminate noise with high probability, but this
# results in a visually over-smooth appearance.  Repeat, specifying a reduction
# in the threshold by factors of 2 and 4.
im_visushrink2 = denoise_wavelet(noisy, channel_axis=-1, convert2ycbcr=True,
                                 method='VisuShrink', mode='soft',
                                 sigma=sigma_est/2, rescale_sigma=True)
im_visushrink4 = denoise_wavelet(noisy, channel_axis=-1, convert2ycbcr=True,
                                 method='VisuShrink', mode='soft',
                                 sigma=sigma_est/4, rescale_sigma=True)

# Compute PSNR as an indication of image quality
psnr_noisy = peak_signal_noise_ratio(original, noisy)
psnr_bayes = peak_signal_noise_ratio(original, im_bayes)
psnr_visushrink = peak_signal_noise_ratio(original, im_visushrink)
psnr_visushrink2 = peak_signal_noise_ratio(original, im_visushrink2)
psnr_visushrink4 = peak_signal_noise_ratio(original, im_visushrink4)

ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title(f'Noisy\nPSNR={psnr_noisy:0.4g}')
ax[0, 1].imshow(im_bayes)
ax[0, 1].axis('off')
ax[0, 1].set_title(
    f'Wavelet denoising\n(BayesShrink)\nPSNR={psnr_bayes:0.4g}')
ax[0, 2].imshow(im_visushrink)
ax[0, 2].axis('off')
ax[0, 2].set_title(
    'Wavelet denoising\n(VisuShrink, $\\sigma=\\sigma_{est}$)\n'
     'PSNR=%0.4g' % psnr_visushrink)
ax[1, 0].imshow(original)
ax[1, 0].axis('off')
ax[1, 0].set_title('Original')
ax[1, 1].imshow(im_visushrink2)
ax[1, 1].axis('off')
ax[1, 1].set_title(
    'Wavelet denoising\n(VisuShrink, $\\sigma=\\sigma_{est}/2$)\n'
     'PSNR=%0.4g' % psnr_visushrink2)
ax[1, 2].imshow(im_visushrink4)
ax[1, 2].axis('off')
ax[1, 2].set_title(
    'Wavelet denoising\n(VisuShrink, $\\sigma=\\sigma_{est}/4$)\n'
     'PSNR=%0.4g' % psnr_visushrink4)
fig.tight_layout()

plt.show()

Estimated Gaussian noise standard deviation = 0.038506558393196055


In [28]:
import matplotlib.pyplot as plt

from skimage import data, filters

# Gaussian filter the image with a sigma of 4.
from scipy.ndimage import gaussian_filter

image =  ggems_projections[0].astype(np.float32)
image = gaussian_filter(image, sigma=1)

image -= image.min()
image /= image.max()

# cutoff frequencies as a fraction of the maximum frequency
cutoffs = [.02, .08, .16]


def get_filtered(image, cutoffs, squared_butterworth=True, order=3.0, npad=0):
    """Lowpass and highpass butterworth filtering at all specified cutoffs.

    Parameters
    ----------
    image : ndarray
        The image to be filtered.
    cutoffs : sequence of int
        Both lowpass and highpass filtering will be performed for each cutoff
        frequency in `cutoffs`.
    squared_butterworth : bool, optional
        Whether the traditional Butterworth filter or its square is used.
    order : float, optional
        The order of the Butterworth filter

    Returns
    -------
    lowpass_filtered : list of ndarray
        List of images lowpass filtered at the frequencies in `cutoffs`.
    highpass_filtered : list of ndarray
        List of images highpass filtered at the frequencies in `cutoffs`.
    """

    lowpass_filtered = []
    highpass_filtered = []
    for cutoff in cutoffs:
        lowpass_filtered.append(
            filters.butterworth(
                image,
                cutoff_frequency_ratio=cutoff,
                order=order,
                high_pass=False,
                squared_butterworth=squared_butterworth,
                npad=npad,
            )
        )
        highpass_filtered.append(
            filters.butterworth(
                image,
                cutoff_frequency_ratio=cutoff,
                order=order,
                high_pass=True,
                squared_butterworth=squared_butterworth,
                npad=npad,
            )
        )
    return lowpass_filtered, highpass_filtered


def plot_filtered(lowpass_filtered, highpass_filtered, cutoffs):
    """Generate plots for paired lists of lowpass and highpass images."""
    fig, axes = plt.subplots(2, 1 + len(cutoffs), figsize=(12, 8))
    fontdict = dict(fontsize=14, fontweight='bold')

    axes[0, 0].imshow(image, cmap='gray')
    axes[0, 0].set_title('original', fontdict=fontdict)
    axes[1, 0].set_axis_off()

    for i, c in enumerate(cutoffs):
        axes[0, i + 1].imshow(lowpass_filtered[i], cmap='gray')
        axes[0, i + 1].set_title(f'lowpass, c={c}', fontdict=fontdict)
        axes[1, i + 1].imshow(highpass_filtered[i], cmap='gray')
        axes[1, i + 1].set_title(f'highpass, c={c}', fontdict=fontdict)

    for ax in axes.ravel():
        ax.set_xticks([])
        ax.set_yticks([])
    plt.tight_layout()
    return fig, axes


# Perform filtering with the (squared) Butterworth filter at a range of
# cutoffs.
lowpasses, highpasses = get_filtered(image, cutoffs, squared_butterworth=True)

fig, axes = plot_filtered(lowpasses, highpasses, cutoffs)
titledict = dict(fontsize=18, fontweight='bold')
fig.text(0.5, 0.95, '(squared) Butterworth filtering (order=3.0, npad=0)',
         fontdict=titledict, horizontalalignment='center')denoise_wavelet?

Signature:
denoise_wavelet(
    image,
    sigma=None,
    wavelet='db1',
    mode='soft',
    wavelet_levels=None,
    convert2ycbcr=False,
    method='BayesShrink',
    rescale_sigma=True,
    *,
    channel_axis=None,
)
Docstring:
Perform wavelet denoising on an image.

Parameters
----------
image : ndarray ([M[, N[, ...P]][, C]) of ints, uints or floats
    Input data to be denoised. `image` can be of any numeric type,
    but it is cast into an ndarray of floats for the computation
    of the denoised image.
sigma : float or list, optional
    The noise standard deviation used when computing the wavelet detail
    coefficient threshold(s). When None (default), the noise standard
    deviation is estimated via the method in [2]_.
wavelet : string, optional
    The type of wavelet to perform and can be any of the options
    ``pywt.wavelist`` outputs. The default is `'db1'`. For example,
    ``wavelet`` can be any of ``{'db2', 'haar', 'sym9'}`` and many more.
mode : {'soft', 'hard'}

In [30]:
import cv2

def prepare(input):
    # preprocessing the image input
    clean = cv2.fastNlMeansDenoising(input)
    ret, tresh = cv2.threshold(clean, 127, 1, cv2.THRESH_BINARY_INV)
    img = crop(tresh)

    # 40x10 image as a flatten array
    flatten_img = cv2.resize(img, (40, 10), interpolation=cv2.INTER_AREA).flatten()

    # resize to 400x100
    resized = cv2.resize(img, (400, 100), interpolation=cv2.INTER_AREA)
    columns = np.sum(resized, axis=0)  # sum of all columns
    lines = np.sum(resized, axis=1)  # sum of all lines

    h, w = img.shape
    aspect = w / h

    return [*flatten_img, *columns, *lines, aspect] 



ModuleNotFoundError: No module named 'cv2'

In [9]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt


In [33]:

# img = cv.fromarray(np.array(ggems_projections[0]))
dst = cv.fastNlMeansDenoising(np.array(ggems_projections[0],dtype = np.uint8),dst=None,h=60,templateWindowSize=8,searchWindowSize=64)
plt.figure()
plt.subplot(121),plt.imshow(np.mean(ggems_projections,axis=0))
plt.subplot(122),plt.imshow(dst)
plt.show()

In [38]:
plt.figure()
# Initialize a heatmap slicer with the mean of the projection data and the denoised data
pli.heatmap_slicer(range(0,ggems_projections[0].shape[0]),range(0,ggems_projections[0].shape[1]),[np.mean(ggems_projections,axis=0),dst],cmap='jet',vmax=np.percentile(np.array(ggems_projections),99),vmin=np.percentile(np.array(ggems_projections),1))

(<Figure size 1800x900 with 3 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>], dtype=object))

[2023-12-04 11:07:37,298] {font_manager.py:1411} DEBUG - findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=12.0.
[2023-12-04 11:07:37,299] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizTwoSymReg.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
[2023-12-04 11:07:37,299] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizOneSymBol.ttf', name='STIXSizeOneSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
[2023-12-04 11:07:37,299] {font_manager.py:1423} DEBUG - findfont: score(FontEntry(fname='/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/matplotlib/mpl-data/fonts

In [65]:
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Flatten each of the projection images and add them to the X array
X = []
for ggems_projection in ggems_projections:
    X.append(ggems_projection.flatten())

X = MinMaxScaler().fit_transform(X)

from sklearn.decomposition import PCA, KernelPCA

kernel_pca = KernelPCA(
    n_components=10,
    kernel="rbf",
    gamma=1e-3,
    fit_inverse_transform=True,
    alpha=5e-3,
    random_state=42,
)

_ = kernel_pca.fit(X_train_noisy)

X_reconstructed_kernel_pca = kernel_pca.inverse_transform(
    kernel_pca.transform(X_test_noisy)
)

Y = X_reconstructed_kernel_pca[0].reshape(ggems_projections[0].shape)

# Rescale Y to its original range

# Plot the ggems projections
plt.figure()
pli.heatmap_slicer(range(0,ggems_projections[0].shape[0]),range(0,ggems_projections[0].shape[1]),[np.mean(ggems_projections,axis=0),Y],cmap='jet',vmax=np.percentile(np.array(ggems_projections),99),vmin=np.percentile(np.array(ggems_projections),1))

/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


(<Figure size 1800x900 with 3 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>], dtype=object))

In [60]:
import logging

import matplotlib.pyplot as plt
from numpy.random import RandomState

from sklearn import cluster, decomposition
from sklearn.datasets import fetch_olivetti_faces

rng = RandomState(0)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

faces = X
n_samples, n_features = faces.shape

# Global centering (focus on one feature, centering all samples)
faces_centered = faces - faces.mean(axis=0)

# Local centering (focus on one sample, centering all features)
faces_centered -= faces_centered.mean(axis=1).reshape(n_samples, -1)

print("Dataset consists of %d faces" % n_samples)

n_row, n_col = 2, 3
n_components = n_row * n_col
image_shape = (512,512)


def plot_gallery(title, images, n_col=n_col, n_row=n_row, cmap=plt.cm.gray):
    fig, axs = plt.subplots(
        nrows=n_row,
        ncols=n_col,
        figsize=(2.0 * n_col, 2.3 * n_row),
        facecolor="white",
        constrained_layout=True,
    )
    fig.set_constrained_layout_pads(w_pad=0.01, h_pad=0.02, hspace=0, wspace=0)
    fig.set_edgecolor("black")
    fig.suptitle(title, size=16)
    for ax, vec in zip(axs.flat, images):
        vmax = max(vec.max(), -vec.min())
        im = ax.imshow(
            vec.reshape(image_shape),
            cmap=cmap,
            interpolation="nearest",
            vmin=-vmax,
            vmax=vmax,
        )
        ax.axis("off")

    fig.colorbar(im, ax=axs, orientation="horizontal", shrink=0.99, aspect=40, pad=0.01)
    plt.show()

pca_estimator = decomposition.KernelPCA(
    n_components=n_components
)
pca_estimator.fit(faces_centered)
plot_gallery(
    "Eigenfaces - PCA using randomized SVD", pca_estimator.eigenvectors_[:n_components]
)

Dataset consists of 90 faces


/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


ValueError: cannot reshape array of size 6 into shape (512,512)

In [64]:
pca_estimator.

1.0

In [15]:
from cv2 import fastNlMeansDenoising

In [19]:
fastNlMeansDenoising?

Docstring:
fastNlMeansDenoising(src[, dst[, h[, templateWindowSize[, searchWindowSize]]]]) -> dst
.   @brief Perform image denoising using Non-local Means Denoising algorithm
.   <http://www.ipol.im/pub/algo/bcm_non_local_means_denoising/> with several computational
.   optimizations. Noise expected to be a gaussian white noise
.   
.   @param src Input 8-bit 1-channel, 2-channel, 3-channel or 4-channel image.
.   @param dst Output image with the same size and type as src .
.   @param templateWindowSize Size in pixels of the template patch that is used to compute weights.
.   Should be odd. Recommended value 7 pixels
.   @param searchWindowSize Size in pixels of the window that is used to compute weighted average for
.   given pixel. Should be odd. Affect performance linearly: greater searchWindowsSize - greater
.   denoising time. Recommended value 21 pixels
.   @param h Parameter regulating filter strength. Big h value perfectly removes noise but also
.   removes image details, small